<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/capstone_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Capstone dataset/Images-out/train'
valid_path = '/content/drive/MyDrive/Capstone dataset/Images-out/test'

In [3]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  


In [18]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(2, activation='softmax')(x)

In [19]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [20]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=180,
                                   zoom_range=0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [26]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Capstone dataset/Images-out/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Capstone dataset/Images-out/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 490 images belonging to 2 classes.
Found 57 images belonging to 2 classes.


In [27]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=15,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-27-694816f4f14b>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/15
31/31 [==============================] - 320s 10s/step - loss: 0.7498 - accuracy: 0.6408 - val_loss: 0.6276 - val_accuracy: 0.7018
Epoch 2/15
31/31 [==============================] - 320s 10s/step - loss: 0.5591 - accuracy: 0.7327 - val_loss: 0.5870 - val_accuracy: 0.6842
Epoch 3/15
31/31 [==============================] - 321s 10s/step - loss: 0.5680 - accuracy: 0.7286 - val_loss: 0.5341 - val_accuracy: 0.7193
Epoch 4/15
31/31 [==============================] - 320s 10s/step - loss: 0.5645 - accuracy: 0.7224 - val_loss: 0.6051 - val_accuracy: 0.6316
Epoch 5/15
31/31 [==============================] - 310s 10s/step - loss: 0.5608 - accuracy: 0.7143 - val_loss: 0.5171 - val_accuracy: 0.7368
Epoch 6/15
31/31 [==============================] - 314s 10s/step - loss: 0.5511 - accuracy: 0.7224 - val_loss: 0.5102 - val_accuracy: 0.7895
Epoch 7/15
31/31 [==============================] - 322s 10s/step - loss: 0.5304 - accuracy: 0.7327 - val_loss: 0.5390 - val_accuracy: 0.7368
Epoch 

In [28]:
import tensorflow as tf

from keras.models import load_model

model.save('scanpath_features_new_model.h5')

In [30]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(vgg.layers))

Number of layers in the base model:  19


In [31]:
# Fine-tune from this layer onwards
fine_tune_at = 100

In [32]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in vgg.layers[:fine_tune_at]:
  layer.trainable = False

In [34]:
base_learning_rate = 0.0001

In [35]:
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
  metrics=['accuracy']
)

In [36]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [38]:
r_fine = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  initial_epoch=r.epoch[-1],
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-38-9f0fce5f64e2>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r_fine = model.fit_generator(


Epoch 15/25
31/31 [==============================] - 328s 11s/step - loss: 0.4856 - accuracy: 0.7776 - val_loss: 0.4917 - val_accuracy: 0.7719
Epoch 16/25
31/31 [==============================] - 315s 10s/step - loss: 0.5051 - accuracy: 0.7531 - val_loss: 0.4891 - val_accuracy: 0.7895
Epoch 17/25
31/31 [==============================] - 313s 10s/step - loss: 0.4798 - accuracy: 0.7776 - val_loss: 0.4904 - val_accuracy: 0.7895
Epoch 18/25
31/31 [==============================] - 315s 10s/step - loss: 0.4859 - accuracy: 0.7837 - val_loss: 0.4895 - val_accuracy: 0.7895
Epoch 19/25
31/31 [==============================] - 325s 10s/step - loss: 0.4987 - accuracy: 0.7531 - val_loss: 0.4904 - val_accuracy: 0.7895
Epoch 20/25
31/31 [==============================] - 316s 10s/step - loss: 0.5067 - accuracy: 0.7592 - val_loss: 0.4909 - val_accuracy: 0.7895
Epoch 21/25
31/31 [==============================] - 320s 10s/step - loss: 0.4969 - accuracy: 0.7694 - val_loss: 0.4901 - val_accuracy: 0.7895

In [40]:
loss, accuracy = model.evaluate(test_set)
print('Test accuracy :', accuracy)

4/4 [==============================] - 32s 8s/step - loss: 0.4893 - accuracy: 0.7895
Test accuracy : 0.7894737124443054
